<a href="https://colab.research.google.com/github/ashmcmn/brain_MRI_estimations/blob/master/One_Component_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# dependencies
!pip install nibabel
!pip install nilearn
!pip install transforms3d
import nibabel as nb
import numpy as np
from matplotlib import pyplot as plt
from nilearn import plotting as nlp
from pandas import concat
import transforms3d
from scipy import ndimage as ndi
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/My Drive/3rd Year/FYP/slice4_ASE.nii.gz" "/content/slice4_ASE.nii.gz"
!yes n | gunzip "/content/slice4_ASE.nii.gz"

In [ ]:
nii = nb.load('slice4_ASE.nii')
dim = nii.header.get_data_shape()
data = nii.get_fdata()
print(data)
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.ravel()
for i in range(10):
  axes[i].imshow(np.squeeze(data)[:,:,i], cmap='Greys_r')

In [ ]:
# load neuroimaging data
nii = nb.load('slice4_ASE.nii')
debug = False

dim = nii.header.get_data_shape()

# defining constants
hct = 0.34; # Expected proportion of hematocrit(volume percentage of red blood cells in blood)
dChi0 = 0.264*10**-6; # Difference in magnetic susceptibility between the tissue and deoxyhaemoglobin contained within blood vessels
gamma = 2.675*10**4; # Proton gyromagnetic ratio
B0 = 3*104; # Strength of main magnetic field
phi = 1.34; # Oxyphoric capacitiy in mlO2/gHb
k = 0.03; # Conversion factor

tau = list(range(-16, 65, 8)) # tau is inter-pulse time (τ)	-16 to 64ms (Δ4)
tau = np.array([x *10**-3 for x in tau]) # converting tau to seconds
print(tau)
ln_Sase = np.log(data)

Tc = 0.016 # characteristic time
tau_lineID = np.where(tau > Tc)[0] # get indices of tau timings where greater than the characteristic time Tc
print(tau_lineID)
w = 1. / np.take(tau, tau_lineID).T # create weightings for the least squares method
p = np.zeros((dim[0],dim[1],dim[2],2))

for i in range(dim[0]): # loop through x coord
  for j in range(dim[1]): # loop through y coord
    for k in range(dim[2]): # loop through z coord to give voxel
      X = tau[tau_lineID] # x coordinates are the time since excitation pulse
      Y = ln_Sase[i,j,k,tau_lineID]  # y coordinates are the magnetic signal strength via the BOLD response (-12 due to regime splitting already done)

      # apply weights to X and Y, transpose data ready for regression
      A = np.vstack([X, np.ones(len(X))]).T
      W = np.sqrt(np.diag(np.array(w).flatten()))
      Aw = np.dot(W, A)
      Yw = np.dot(Y.T, W)

      p[i, j, k,:] = np.linalg.lstsq(Aw, Yw.T, rcond=1e-10)[0] # find the least-squares solution for the x and y coordinates
      m, c = p[i, j, k,:]
      if debug and m == m:
        _ = plt.plot(X, Y, 'o', label='Original data', markersize=10)
        _ = plt.plot(X, m*tau[tau_lineID] + c, 'r', label='Fitted line')
        plt.xlabel('Time since excitation pulse')
        plt.ylabel('Signal strength')
        _ = plt.legend()
        plt.show()
      
      

range_r2p = np.array([0, 7])
range_dbv = np.array([0, 0.12])
range_oef = np.array([0, 1.0])
range_dhb = np.array([0, 10])

s0_id = np.where(tau == 0)

r2p = -p[:,:,:,0]
c = p[:,:,:,1]

dbv = c - ln_Sase[:,:,:,s0_id]
oef = r2p/(dbv * gamma * (4/3) * dChi0 * hct * B0)
dhb = r2p/(dbv * gamma * (4/3) * dChi0 * B0 * k)